In [4]:

import ee
import geemap
from corvus_py.landsat import basic_processing
from corvus_py.landsat import collection_building
from corvus_py.phenology_extraction import cloud_temporal_filter
from corvus_py.phenology_extraction import phenology_extraction

ee.Initialize(project='corvus-phenology-drylands')

col_example = (collection_building.get_collection('2024-01-01', '2024-12-31', 1).filterBounds(ee.Geometry.Point([-119.5, 34.5]))
                                  .map(basic_processing.get_ndvi_457)
                                  .map(collection_building.add_doy)
                                  .map(collection_building.add_time))
display(col_example)

col_filt = cloud_temporal_filter.cloud_temporal_filter(col_example, "NDVI", -0.1, 10000000, 10)

display(col_filt)

col_smoothed = phenology_extraction.fit_phenology(col_filt, 30, 30, 52, "NDVI", ee.Date('2024-01-01'), ee.Date('2024-12-31'), min_value=-0.2, max_value=1.0)
display(col_smoothed)

Map = geemap.Map(center=[34.5, -119.5], zoom=10) 
Map.addLayer(col_example, {'bands': ['SR_B3', 'SR_B2', 'SR_B1'], 'min': 0, 'max': 0.2}, 'Image Example', False)
Map.addLayer(col_example.select('NDVI').count(), {'min': 20, 'max': 35}, 'NDVI Count', False)
Map.addLayer(col_example.select('NDVI').first(), {'min': 0, 'max': 1}, 'NDVI Example', False)
Map.addLayer(ee.Image((col_filt.toList(col_filt.size())).get(30)).select('previous_DOY'), {'min': -10, 'max': 10}, 'Previous DOY', False)
Map.addLayer(ee.Image((col_filt.toList(col_filt.size())).get(30)).select('next_DOY'), {'min': -10, 'max': 10}, 'Next DOY', False)
Map.addLayer(ee.Image((col_filt.toList(col_filt.size())).get(30)).select('DOY'), {'min': -10, 'max': 10}, 'Current DOY', False)
Map.addLayer(col_filt.select('previous').first(), {'min': 0, 'max': 1}, 'NDVI Previous', False)
Map.addLayer(col_filt.select('next').first(), {'min': 0, 'max': 1}, 'NDVI Next', False)
Map.addLayer(col_filt.select('NDVI').count(), {'min': 20, 'max': 35}, 'NDVI Filtered Count', False)
Map.addLayer(col_filt.select('reg_diff').first(), {'min': -0.2, 'max': 0.2}, 'NDVI Filtered Differences', True)
Map.addLayer(col_filt.select('reg_expected').first(), {'min': 0, 'max': 1}, 'NDVI Filtered Expected', True)
Map.addLayer(col_filt.select('NDVI').toBands(), {'min': 20, 'max': 35}, 'NDVI Filtered', False)
Map.addLayer(col_smoothed.select('prediction_filtered').toBands(), {'bands': ['40_prediction_filtered', '20_prediction_filtered', '4_prediction_filtered'], 'min': 0.3, 'max': 0.8}, 'NDVI Phenology', False)
Map

Map(center=[34.5, -119.5], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…